In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import shuffle
from sklearn.model_selection import train_test_split
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
SEED=2
import os

train=pd.read_csv("/kaggle/input/aptos2019-blindness-detection/train.csv")
test=pd.read_csv("/kaggle/input/aptos2019-blindness-detection/test.csv")

x = train['id_code']
y = train['diagnosis']


# Any results you write to the current directory are saved as output.

In [2]:
X_train,X_test,y_train,y_test=train_test_split(x, y, test_size=0.15)


In [3]:
def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [4]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [5]:
import matplotlib.pyplot as plt 
import cv2
train_images=[]
IMG_SIZE=277
%time
fig = plt.figure(figsize=(25, 16))

for idx, row in enumerate(X_train):
    
    path=f"../input/aptos2019-blindness-detection/train_images/{row}.png"
    image =load_ben_color(path)
    train_images.append(image)

        

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.58 µs


<Figure size 1800x1152 with 0 Axes>

In [6]:
import matplotlib.pyplot as plt 
import cv2
test_images=[]
IMG_SIZE=277
%time
fig = plt.figure(figsize=(25, 16))

for idx, row in enumerate(X_test):
    
    path=f"../input/aptos2019-blindness-detection/train_images/{row}.png"
    image =load_ben_color(path)
    test_images.append(image)


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.58 µs


<Figure size 1800x1152 with 0 Axes>

In [7]:

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D)
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [8]:
function = "softmax"
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = ResNet50(include_top=False,
                   weights=None,
                   input_tensor=input_tensor)
    base_model.load_weights('../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation=function, name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model

In [9]:
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.applications.resnet50 import preprocess_input
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score, cohen_kappa_score, accuracy_score
from keras.utils import Sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

WORKERS = 2
CHANNEL = 3

import warnings
warnings.filterwarnings("ignore")
SIZE = 300
NUM_CLASSES = 5

In [10]:
# create callbacks list
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau,CSVLogger)

epochs = 30; batch_size = 32
checkpoint = ModelCheckpoint('../working/Resnet50.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, 
                                   verbose=1, mode='min', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=9)

csv_logger = CSVLogger(filename='../working/training_log.csv',
                       separator=',',
                       append=True)
# callbacks_list = [checkpoint, csv_logger, reduceLROnPlat, early]


model = create_model(
    input_shape=(SIZE,SIZE,3), 
    n_out=NUM_CLASSES)

OSError: Unable to open file (unable to open file: name = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [11]:
train_files =load_dataset(trainPath,train['id_code'])
y_train=[]
for y in train['diagnosis']:
    y_train.append(y)


NameError: name 'load_dataset' is not defined

In [12]:
test_files =load_dataset(testPath,test['id_code'])

y_test=[]

for y in submission['diagnosis']:
    y_test.append(y)

NameError: name 'load_dataset' is not defined

In [13]:
y_train_dummies=pd.get_dummies(y_train)
y_test_dummies=pd.get_dummies(y_test)


In [14]:
from keras.preprocessing import image                  
from tqdm import tqdm

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense,BatchNormalization,Activation
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint  
from keras.applications import *




In [15]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224),color_mode ="grayscale")
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return  np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [16]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255

test_tensors = paths_to_tensor(test_files).astype('float32')/255

NameError: name 'train_files' is not defined

In [17]:
train_tensors.shape[1:]

NameError: name 'train_tensors' is not defined

In [18]:
dim = 256
from keras import backend as K
model = Sequential()


#trainImages=np.expand_dims(trainImages, axis=1)
model.add(Conv2D(16,(dim,dim),padding='same',input_shape=train_tensors.shape[1:]))
model.add(MaxPooling2D(pool_size=2,strides=2,dim_ordering="th"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv2D(32,(3,3),padding='same'))
model.add(MaxPooling2D(pool_size=2, strides=2,dim_ordering="th"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=2, strides=2,dim_ordering="th"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(GlobalAveragePooling2D())

model.add(Dense(5,activation='softmax'))
model.summary()

NameError: name 'train_tensors' is not defined

In [19]:
np.expand_dims(train_tensors.shape,axis=1)

NameError: name 'train_tensors' is not defined

In [20]:
epochs = 100
learning_rate = 0.0001
batch_size = 32

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# ------ TRAINING ------
model.fit(train_tensors , y_train_dummies, epochs=30,batch_size=20, verbose=1)
                    

NameError: name 'train_tensors' is not defined

In [22]:
model_prediction = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_tensors]



NameError: name 'test_tensors' is not defined

In [23]:
from sklearn.metrics import fbeta_score, make_scorer

accuracy= fbeta_score(y_test,model_prediction,beta=2)


print(accuracy)



NameError: name 'model_prediction' is not defined